In [40]:
import json
import pandas as pd
import os

In [41]:
def load_jsons():
    
    routing_input_path = 'routing_input.json'
    routing_output_path = 'routing_output.json'
    
    with open(routing_input_path, 'r') as routing_input:
        routingInput = json.load(routing_input)
        
    with open(routing_output_path, 'r') as routing_output:
        routingOutput = json.load(routing_output)
        
    return routingInput, routingOutput

In [42]:
def allowed_sites_for_vehicles(sites_df, vehicles_df):
    
    allowed_sites = [[],[],[],[],[],[],[],[],[]]
    
    vehicles_df['allowed_sites'] = allowed_sites
    
    for index1, row1 in sites_df.iterrows():
        for index2, row2 in vehicles_df.iterrows():
            if sites_df.iloc[index1]["type"] == vehicles_df.iloc[index2]["type"]:
                vehicles_df.iloc[index2]["allowed_sites"].append(sites_df.iloc[index1]["location_index"])
    
    return vehicles_df

In [43]:
def filter_wrong_visits_from_routes(vehicles_df, routes_df):
    
    for index1, row1 in routes_df.iterrows():
        for index2, row2 in vehicles_df.iterrows():
            filtered = []
            for location in routes_df.iloc[index1]["vehicles"][index2]['route']:
                                
                if location in vehicles_df.iloc[index2]["allowed_sites"]:
                    filtered.append(location)
                
                elif location == 394 or location == 395:
                    filtered.append(location)
            
            if len(filtered) > 2:
                routes_df.iloc[index1]["vehicles"][index2]['route'] = filtered
    
            else:
                routes_df.iloc[index1]["vehicles"][index2]['route'] = []
    
    return routes_df

In [44]:
def main():
    
    routing_input, routing_output = load_jsons()
    
    pickup_sites = routing_input["pickup_sites"]
    pickup_sites_df = pd.DataFrame(pickup_sites)[["location_index", "type"]]
    
    vehicles = routing_input["vehicles"]
    vehicles_df = pd.DataFrame(vehicles)[["type"]]
    
    routes = routing_output["days"]
    routes_df = pd.DataFrame(routes)    
   
    vehicles_df = allowed_sites_for_vehicles(pickup_sites_df, vehicles_df)
    
    routes_df = filter_wrong_visits_from_routes(vehicles_df, routes_df)
    
    return vehicles_df, pickup_sites_df, routes_df 
        

In [45]:
vehicles, sites, routes = main()

In [46]:
vehicles

,type,allowed_sites
0,3,"[32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 8..."
1,1,"[191, 192, 193, 194, 195, 196, 197, 198, 199, ..."
2,2,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
3,3,"[32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 8..."
4,2,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
5,1,"[191, 192, 193, 194, 195, 196, 197, 198, 199, ..."
6,3,"[32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 8..."
7,1,"[191, 192, 193, 194, 195, 196, 197, 198, 199, ..."
8,2,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."


In [47]:
routes

,vehicles
0,"[{'route': []}, {'route': []}, {'route': []}, ..."
1,"[{'route': []}, {'route': []}, {'route': []}, ..."
2,"[{'route': []}, {'route': []}, {'route': []}, ..."
3,"[{'route': []}, {'route': []}, {'route': []}, ..."
4,"[{'route': []}, {'route': []}, {'route': []}, ..."
...,...
246,"[{'route': []}, {'route': []}, {'route': []}, ..."
247,"[{'route': []}, {'route': []}, {'route': []}, ..."
248,"[{'route': []}, {'route': []}, {'route': []}, ..."
249,"[{'route': []}, {'route': []}, {'route': []}, ..."


In [48]:
#routes["days"].iloc[1]['vehicles']

In [49]:
#routes["days"].iloc[2]['vehicles']

In [50]:
sites

,location_index,type
0,0,2
1,1,2
2,2,2
3,3,2
4,4,2
...,...,...
389,389,1
390,390,1
391,391,1
392,392,1


In [51]:
routes

,vehicles
0,"[{'route': []}, {'route': []}, {'route': []}, ..."
1,"[{'route': []}, {'route': []}, {'route': []}, ..."
2,"[{'route': []}, {'route': []}, {'route': []}, ..."
3,"[{'route': []}, {'route': []}, {'route': []}, ..."
4,"[{'route': []}, {'route': []}, {'route': []}, ..."
...,...
246,"[{'route': []}, {'route': []}, {'route': []}, ..."
247,"[{'route': []}, {'route': []}, {'route': []}, ..."
248,"[{'route': []}, {'route': []}, {'route': []}, ..."
249,"[{'route': []}, {'route': []}, {'route': []}, ..."


In [52]:
routes.to_json('routing_output_without_wrong_visits.json', orient = 'records')